In [ ]:
import os
import requests
from dotenv import load_dotenv
import base64
import json
import numpy as np
import pandas as pd
from snowflake.connector.pandas_tools import write_pandas
from utils.sf_conn import Config
from utils.assistant_api import Spotify_Assistant
from utils.prompt import PROMPT
import time
load_dotenv()

In [ ]:
sf_config = Config()
create_conn = sf_config.create_sf_conn()

In [ ]:
cursor = create_conn.cursor()
query = "SELECT * FROM MAIN_SPOTIFY"
spotify_df = cursor.execute(query).fetch_pandas_all()
spotify_df.head(3)

In [ ]:
spotify_csv = spotify_df.to_csv('assistant_data/spotify_main_file')

In [ ]:
def wait_until_run_status_completed(assistant, run_id):
    start_time = time.time() # starting a timer in case, the while loop gets stuck
    while True:
        (status := assistant.run_status(run_id))
        if status == "completed":
            break
        elif status in {"in_progress", "queued"}:
            cur_time = time.time()
            time_elasped = cur_time - start_time
            if time_elasped >= 600: # break the loop after it's been running for 10 mins
                break
            continue
        else:
            break
    assert status == "completed"

In [ ]:
# def upload_csv_file(file_path):
#     api_key =  os.getenv('OPENAI_API_KEY')
#     openai.api_key = api_key
#     with open(file_path, "rb") as file:
#         uploaded_file = openai.files.create(
#             file=file,
#             purpose='assistants'
#         )
#     return uploaded_file.id

In [ ]:
assistant = Spotify_Assistant(assistant_id='asst_afQAdlE40WllyVRI7mDZBjha')
upload_csv = assistant.upload_csv_file('assistant_data/spotify_main_file')
print("------------upload_csv--------------------------------")
print(upload_csv)
files = assistant.show_filed_uplaoded()
print("---------Show Files in Assistant-------------------")
print(files)
print("---------Delete Files in assistant---------------------")
# delete_files = assistant.delete_file()
# print(delete_files)


In [ ]:
delete_files = assistant.delete_file()
delete_files

In [ ]:
for i in files:
    print(i.id)

In [ ]:
! pip list

In [ ]:
import os

import snowflake.connector as sfc
from snowflake.connector.pandas_tools import write_pandas
import json



def create_sf_conn(
            wh='COMPUTE_WH',
            r='DATA_ENGINEER',
            keep_alive=False):
    conn = sfc.connect(
            user = os.getenv("SF_USERNAME"),
            password = os.getenv("SF_PASSWORD"),
            account = os.getenv('SF_ACCOUNT'),
            warehouse = wh,
            role = r,
            database = 'analytics_justin', 
            schema = 'spotify_schema', 
            client_session_keep_alive=keep_alive)
    return conn

In [ ]:
con = create_sf_conn()
cursor = con.cursor()
execute = cursor.execute('SELECT * from main_spotify')
execute.fetch_pandas_all()


In [3]:
import os
import requests
from dotenv import load_dotenv
load_dotenv()
import base64
import json
import numpy as np
import pandas as pd
import pyarrow
from snowflake.connector.pandas_tools import write_pandas
from utils.sf_conn import Config
from json.decoder import JSONDecodeError
import time

### these are the api keys that we will use
client_id = os.getenv('CLIENT_ID')
client_secret = os.getenv('CLIENT_SECRET')

### This is going to be the playlists that we are extracting data from
spotify_playlist_ids = ['37i9dQZF1EQp9BVPsNVof1?si=b0e0182f48ea461c'
                "37i9dQZF1EIh5VHows1LoY?si=3f242c7e9e7a43f8",
                '37i9dQZF1EVKuMoAJjoTIw?si=2ed6df3d293b4519',
                '37i9dQZF1EQpoj8u9Hn81e?si=92566118c4954e73',
                '37i9dQZF1EQqlvxWrOgFZm?si=2b9ab93cc7714708',
                '37i9dQZF1EIhBF9gUW4UsT?si=089940507c6b46f3',
                '37i9dQZF1EIherXksVvnrN?si=d8e28d16e28b40b0',
                '37i9dQZF1EVHGWrwldPRtj?si=98356d3db1994d67',
                "37i9dQZF1DWSf2RDTDayIx?si=1d3f916447094372",
                "37i9dQZF1DX0AMssoUKCz7?si=80092935821a4201",
                '37i9dQZF1DX0hvSv9Rf41p?si=72b7f2562f0a4020',
                "6FSFUWzuF2KigQDKV42Uru?si=cfc88421610b46c3",
                '37i9dQZF1E35DbZEmy06Qp?si=b6e38bde3d8d4116',
                '37i9dQZF1EIgtj4OvJCT7Q?si=d18338df81cc4ff1']

# Encode Client ID and Client Secret
client_creds = f"{client_id}:{client_secret}"
client_creds_b64 = base64.b64encode(client_creds.encode()).decode()

auth_url = 'https://accounts.spotify.com/api/token'
headers = {
    'Authorization': f"Basic {client_creds_b64}"
}

payload = {
    "grant_type": "client_credentials"
}

auth_response = requests.post(auth_url, headers=headers, data=payload)
auth_response_data = auth_response.json()
# print(auth_response_data)
access_token = auth_response_data['access_token']


In [2]:
def get_playlist_name_tracks(playlist_ids, access_token = access_token):
    all_playlists_data = []
    for ids in playlist_ids:
        # Get the playlist's name
        url = f"https://api.spotify.com/v1/playlists/{ids}"
        headers = {'Authorization': f'Bearer {access_token}'}
        name_response = requests.get(url, headers=headers)
        print(name_response)
        # if name_response.status_code == 200:

        #     data = name_response.json()
        #     return data

In [3]:
data = get_playlist_name_tracks('37i9dQZF1EIgtj4OvJCT7Q?si=d18338df81cc4ff1')
data

<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [404]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>


In [4]:
all_artist = sp.current_user_followed_artists(limit=20, after = None)

In [8]:
all_artist

{'artists': {'items': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/0ycHhPwPvoaO4VGzmMnXGq'},
    'followers': {'href': None, 'total': 758793},
    'genres': ['brostep', 'edm', 'glitch'],
    'href': 'https://api.spotify.com/v1/artists/0ycHhPwPvoaO4VGzmMnXGq',
    'id': '0ycHhPwPvoaO4VGzmMnXGq',
    'images': [{'height': 640,
      'url': 'https://i.scdn.co/image/ab6761610000e5eb5643f477e79c915bc776500a',
      'width': 640},
     {'height': 320,
      'url': 'https://i.scdn.co/image/ab676161000051745643f477e79c915bc776500a',
      'width': 320},
     {'height': 160,
      'url': 'https://i.scdn.co/image/ab6761610000f1785643f477e79c915bc776500a',
      'width': 160}],
    'name': 'Zomboy',
    'popularity': 49,
    'type': 'artist',
    'uri': 'spotify:artist:0ycHhPwPvoaO4VGzmMnXGq'},
   {'external_urls': {'spotify': 'https://open.spotify.com/artist/1ZzI9ohe9Ekq5yZhK3U9pF'},
    'followers': {'href': None, 'total': 15573},
    'genres': [],
    'href': 'https://api.sp

In [6]:
with open('follow_artist_list.json', 'a', encoding= 'utf-8') as file:
    all_artist = sp.current_user_followed_artists(limit=20, after = None)
    artist_info = all_artist['artists']
    json.dump(artist_info, file, ensure_ascii= False)
    file.write('\n')

In [10]:
count = 0
for i in all_artist['artists']['items']:
    artist_name = i['name']
    count +=1
    print(f'{count}:{artist_name}')


1:Zomboy
2:Picture Us Tiny
3:Ganja White Night
4:SLANDER
5:ODESZA
6:Flosstradamus
7:ILLENIUM
8:Logic
9:Phish
10:Aphex Twin


In [1]:
from utils.spotify_api import Spotify
import json

In [4]:
spot = Spotify(access_token=access_token)
playlist_df, track_data = spot.get_playlist_name_tracks(['37i9dQZF1EIgtj4OvJCT7Q?si=d18338df81cc4ff1'])

In [5]:
with open('/Users/justinfarnan_hakkoda/Toll_Brothers/test_json_data.json', 'w') as save_data:
    json.dump(track_data, save_data, indent=6)

In [6]:
# columns = []
# for items in track_data.get('tracks', {}).get('items', []):
#     track = items.get('track')
#     print(track)

In [7]:
# Function to infer SQL data type from Python data type
def infer_sql_type(py_value):
    if isinstance(py_value, int):
        return "INT"
    elif isinstance(py_value, float):
        return "FLOAT"
    elif isinstance(py_value, bool):
        return "BOOLEAN"
    elif isinstance(py_value, str):
        return "VARCHAR(255)"  # You might choose different size based on your data
    else:
        return "TEXT"  # Fallback type

In [9]:
def generate_table_schema(data, table_name):
    ddl = f"CREATE TABLE {table_name} (\n"
    columns = []
    for key, value in data.items():
        if isinstance(value, dict):
            # Suggest a new table for nested dictionaries
            # nested_table_name = f"{table_name}_{key}"
            # print(f"// Consider creating a separate table '{nested_table_name}' for nested object '{key}'")
            columns.append(f"    {key.upper()} VARCHAR(255)")
            for dict_key, dict_value in value.items():
                nest_dict_sql_type = infer_sql_type(dict_value)
                columns.append(f"    {dict_key.upper()} {nest_dict_sql_type}")  # Example: Foreign key reference
        elif isinstance(value, list):
            # print(value[0])
            # # Suggest handling array as separate table or text
            # print(f"// Consider handling array '{key}' separately if it contains complex objects")
            columns.append(f"    {key.upper()} TEXT")
            for list_key, list_value in value[0].items():
                nest_list_sql_type = infer_sql_type(list_value)
                columns.append(f"    {list_key.upper()} {nest_list_sql_type}")  # Could be JSON text or a foreign key list
        else:
            sql_type = infer_sql_type(value)
            columns.append(f"    {key.upper()} {sql_type}")
    ddl += ",\n".join(columns)
    ddl += "\n);"
    return ddl

# Generate SQL for primary table
table_name = "playlist"
ddl_statement = generate_table_schema(data=track_data, table_name= table_name)
print(ddl_statement)



CREATE TABLE playlist (
    COLLABORATIVE INT,
    DESCRIPTION VARCHAR(255),
    EXTERNAL_URLS VARCHAR(255),
    SPOTIFY VARCHAR(255),
    FOLLOWERS VARCHAR(255),
    HREF TEXT,
    TOTAL INT,
    HREF VARCHAR(255),
    ID VARCHAR(255),
    IMAGES TEXT,
    HEIGHT TEXT,
    URL VARCHAR(255),
    WIDTH TEXT,
    NAME VARCHAR(255),
    OWNER VARCHAR(255),
    DISPLAY_NAME VARCHAR(255),
    EXTERNAL_URLS TEXT,
    HREF VARCHAR(255),
    ID VARCHAR(255),
    TYPE VARCHAR(255),
    URI VARCHAR(255),
    PRIMARY_COLOR TEXT,
    PUBLIC INT,
    SNAPSHOT_ID VARCHAR(255),
    TRACKS VARCHAR(255),
    HREF VARCHAR(255),
    ITEMS TEXT,
    LIMIT INT,
    NEXT TEXT,
    OFFSET INT,
    PREVIOUS TEXT,
    TOTAL INT,
    TYPE VARCHAR(255),
    URI VARCHAR(255)
);
